In [51]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

In [52]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')

In [53]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 63.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 59.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 44.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 41.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 59.3MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [54]:

#  Load and display all CV PDFs in order

import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
John Smith

Marketing Professional

+ Singapore, Singapore (cid:209) Kowloon

Experience

Engineer, ByteDance

2020 – Present

• Worked in a fast-paced, global technology environment.

• Collaborated across teams to support large-scale platforms.

• Applied analytical and problem-solving skills in production systems.

Education

McGill University

Bachelor of Science (BSc) in Marketing

Skills

Content Creation

SEO

Social Media

Graduated 2009

1





📄 CV_2.pdf
Minh Pham
Design Professional

Beijing, China | Hong Kong

Professional Experience

Manager, BCG

2022 – Present

• Led cross-functional teams on client-facing design initiatives.

• Managed project timelines, deliverables, and stakeholder communication.

• Applied design thinking to business and strategy problems.

Analyst, Tencent

2013 – 2017

• Conducted market and product analysis to support decision-making.

• Collaborated with design and engineering teams.

• Produced reports and insights for senior leadersh

In [55]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

In [56]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient

In [57]:

# 1. Define a local tool

@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

In [58]:

# 2. Load MCP tools + merge

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

In [59]:
#3. Initialize Gemini (tool-enabled) or deepseek

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GEMINI_VERTEX_API_KEY,
    temperature=0,
)


The GCP project ID required for the next code module is: linen-age-483607-m5

In [65]:
!pip install --upgrade langchain-google-vertexai langchain_mcp_adapters google-cloud-aiplatform


import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_google_vertexai import ChatVertexAI


from google.colab import auth
auth.authenticate_user()

import google.auth
credentials, project_id = google.auth.default()
print(f"The project_id obtained from the default credentials: '{project_id}'")


if not project_id:
    project_id = os.getenv('GOOGLE_CLOUD_PROJECT')
    if not project_id:
        project_id = input("Please enter your GCP project ID: ").strip()
        if not project_id:
            raise ValueError("To use Vertex AI, you must provide the GCP project ID.")
    print(f"Use the project ID: {project_id}")


@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! "


client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

print("List of Available Tools：")
for i, t in enumerate(tools):
    print(f"{i}: {t.name}")


llm = ChatVertexAI(
    model="gemini-2.0-flash",
    credentials=credentials,
    location="us-central1",
    project=project_id,
    temperature=0,
)


llm_with_tools = llm.bind_tools(tools)


query = "Say hello to Bao using tool, then search for someone named Alice on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print("\nModel response：")
print(response)

The project_id obtained from the default credentials: ''
Please enter your GCP project ID: linen-age-483607-m5
Use the project ID: linen-age-483607-m5
List of Available Tools：
0: search_facebook_users
1: get_facebook_profile
2: get_facebook_mutual_friends
3: search_linkedin_people
4: get_linkedin_profile
5: get_linkedin_interactions
6: say_hello


/tmp/ipython-input-1029413418.py:50: DeprecationWarning: Use [`ChatGoogleGenerativeAI`][langchain_google_genai.ChatGoogleGenerativeAI] instead.
  llm = ChatVertexAI(



Model response：
content='' additional_kwargs={'function_call': {'name': 'search_facebook_users', 'arguments': '{"q": "Alice"}'}} response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 2572, 'candidates_token_count': 13, 'total_token_count': 2585, 'prompt_tokens_details': [{'modality': 1, 'token_count': 2572}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 13}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.00012376578524708748, 'model_provider': 'google_vertexai', 'model_name': 'gemini-2.0-flash'} id='lc_run--019c7737-2b77-7372-82d2-561e6c768620-0' tool_calls=[{'name': 'say_hello', 'args': {'name': 'Bao'}, 'id': '978f578e-f61b-44fd-b7b1-ed094b374997', 'type': 'tool_call'}, {'name': 'search_facebook_users', 'args': {'q': 'Alice'}, 'id': '94be6dc8-1b2d-430a-828b-692332a358b7', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'in

In [63]:
llm_with_tools = llm.bind_tools(tools, tool_choice="auto")

In [64]:
query = """
Please help me with two tasks: greet Bao and search for Alice on Facebook.

You must follow this exact response structure:
1. First, output the following sentence verbatim: "I'll help you with that! First, let me say hello to Bao, and then search for Alice on Facebook." Do not add any extra words before or after.
2. Then, call the 'say_hello' tool with parameter {{"name": "Bao"}}.
3. Then, call the 'search_facebook_users' tool with parameter {{"q": "Alice"}}.

Do not explain anything beyond the required sentence. Do not mention the tool names in the sentence.
"""
response = llm_with_tools.invoke([HumanMessage(content=query)])
print(response)

content="I'll help you with that! First, let me say hello to Bao, and then search for Alice on Facebook.\n" additional_kwargs={'function_call': {'name': 'search_facebook_users', 'arguments': '{"q": "Alice"}'}} response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 2699, 'candidates_token_count': 39, 'total_token_count': 2738, 'prompt_tokens_details': [{'modality': 1, 'token_count': 2699}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 39}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.0001792284086919748, 'model_provider': 'google_vertexai', 'model_name': 'gemini-2.0-flash'} id='lc_run--019c7733-2a2f-70d2-b861-83fd3d9c376d-0' tool_calls=[{'name': 'say_hello', 'args': {'name': 'Bao'}, 'id': 'c54d4b39-d83d-41e9-a99a-6ee027ffdb8d', 'type': 'tool_call'}, {'name': 'search_facebook_users', 'args': {'q': 'Alice'}, 'id': '4bba81ad-d26d-4f2c-8922

In [66]:
# Test 1: Search Facebook users with exact match
# This calls the search_facebook_users tool with query "Alex Chan" and limit 5.
# The result is a list of matching user profiles.

result = await tools[0].ainvoke({'q': 'Alex Chan', 'limit': 5})
print("Test 1 - search_facebook_users (exact match):")
print(result)

Test 1 - search_facebook_users (exact match):
[{'type': 'text', 'text': '[{"id":3,"display_name":"Alex Chan","city":"Hyderabad","country":"India","match_type":"exact"},{"id":41,"display_name":"Alex Chan","city":"Shanghai","country":"China","match_type":"exact"},{"id":79,"display_name":"Alex Chan","city":"London","country":"UK","match_type":"exact"},{"id":106,"display_name":"Alex Chan","city":"Ho Chi Minh City","country":"Vietnam","match_type":"exact"},{"id":117,"display_name":"Alex Chan","city":"Melbourne","country":"Australia","match_type":"exact"}]', 'id': 'lc_01d57645-0290-483f-a06e-e4d41eb1d2b4'}]


In [67]:
# Test 2: Search Facebook users with fuzzy matching (handles typos)
# Query "Alx Chn" with fuzzy=True to allow approximate name matching.

result = await tools[0].ainvoke({'q': 'Alx Chn', 'limit': 5, 'fuzzy': True})
print("Test 2 - search_facebook_users (fuzzy match):")
print(result)

Test 2 - search_facebook_users (fuzzy match):
[{'type': 'text', 'text': '[{"id":3,"display_name":"Alex Chan","city":"Hyderabad","country":"India","match_type":"fuzzy"},{"id":41,"display_name":"Alex Chan","city":"Shanghai","country":"China","match_type":"fuzzy"},{"id":79,"display_name":"Alex Chan","city":"London","country":"UK","match_type":"fuzzy"},{"id":106,"display_name":"Alex Chan","city":"Ho Chi Minh City","country":"Vietnam","match_type":"fuzzy"},{"id":117,"display_name":"Alex Chan","city":"Melbourne","country":"Australia","match_type":"fuzzy"}]', 'id': 'lc_232505dc-698c-43d6-bcc1-5fbb2c90fc07'}]


In [68]:
# Test 3: Retrieve a complete Facebook profile by user ID
# Note: Replace 123 with a valid user ID from a previous search.
# If the ID is invalid, the tool will return an error (expected for demonstration).

try:
    result = await tools[1].ainvoke({'user_id': 123})
    print("Test 3 - get_facebook_profile:")
    print(result)
except Exception as e:
    print("Test 3 - get_facebook_profile failed (expected if user_id 123 is invalid):", e)

Test 3 - get_facebook_profile:
[{'type': 'text', 'text': '{"id":123,"display_name":"Emily Davis","original_name":"Emily Davis","city":"London","country":"UK","hometown":"London","bio":"Price audience both east scientist.  Eye beyond kind range event coach any him","status":"Divorced","education":"Master\'s Degree","current_job":"Analyst","current_company":"NextGen AI","interests":"Reading, Wine Tasting, Running, Hiking","friend_count":44,"friends":[5,12,9122,7234,290,9733,9576,9100,7917,9358,3759,6540,8753,243,8950,4214,5110,8470,1630,365,697,1446,1900,2679,2745,2892,3775,3783,4398,4441,4664,4866,5495,5754,6583,6788,7241,7745,7982,8283,9004,9388,9507,9976],"posts":[{"id":689,"content":"Good vibes only ✨"},{"id":690,"content":"Just got back from London. Already planning my next trip! ✈️"},{"id":691,"content":"Concert vibes in London last night! Best live music 🎵"},{"id":692,"content":"First week at NextGen AI complete! Loving the team already 💼"},{"id":693,"content":"Celebrating small w

In [70]:
# Test 4: Find mutual friends between two Facebook users
# Requires two valid user IDs. Replace 123 and 456 with real IDs from search results.
# If IDs are invalid, the tool will return an error.

try:
    result = await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})
    print("Test 4 - get_facebook_mutual_friends:")
    print(result)
except Exception as e:
    print("Test 4 - get_facebook_mutual_friends failed (expected if IDs are invalid):", e)

Test 4 - get_facebook_mutual_friends:
[{'type': 'text', 'text': '{"user_1_id":123,"user_2_id":456,"mutual_friends":[],"mutual_count":0}', 'id': 'lc_fdacb856-2456-4a7d-8e0a-0a1abb2ac812'}]


In [73]:

# Test 5: Search LinkedIn people with exact match
# Query for skill "Python" with location filter "Hong Kong".

result = await tools[3].ainvoke({'q': 'Python', 'location': 'Hong Kong', 'limit': 5})
print("Test 5 - search_linkedin_people (exact match):")
print(result)

Test 5 - search_linkedin_people (exact match):
[{'type': 'text', 'text': '[{"id":95,"name":"Rahul Sharma","headline":"AI Professional","industry":"AI","location":"Kowloon, Hong Kong","years_experience":16,"match_type":"exact"},{"id":226,"name":"Amit Patel","headline":"Software Professional","industry":"Software","location":"Central, Hong Kong","years_experience":4,"match_type":"exact"},{"id":381,"name":"Sophia Ho","headline":"AI Professional","industry":"AI","location":"Sha Tin, Hong Kong","years_experience":11,"match_type":"exact"},{"id":594,"name":"Michael Cheung","headline":"Software Professional","industry":"Software","location":"Causeway Bay, Hong Kong","years_experience":15,"match_type":"exact"},{"id":604,"name":"Amit Patel","headline":"Software Professional","industry":"Software","location":"Central, Hong Kong","years_experience":8,"match_type":"exact"}]', 'id': 'lc_0042e6df-ebc8-438a-b590-fffce9b4f90f'}]


In [72]:
# Test 7: Retrieve a complete LinkedIn profile by person ID
# Replace 456 with a valid person ID from a previous search.
# If ID is invalid, the tool will return an error.

try:
    result = await tools[4].ainvoke({'person_id': 456})
    print("Test 7 - get_linkedin_profile:")
    print(result)
except Exception as e:
    print("Test 7 - get_linkedin_profile failed (expected if person_id 456 is invalid):", e)

Test 7 - get_linkedin_profile:
[{'type': 'text', 'text': '{"id":456,"name":"Michael Cheung","headline":"Legal Professional","city":"San Francisco","country":"USA","industry":"Legal","status":"student","years_experience":1,"summary":"Series I American so. Total though military campaign. Way sing international imagine. Carry evidence and economy model account. Check face physical leave strategy behind. What attorney opportunity stand.","skills":[{"name":"Compliance","proficiency":1},{"name":"Contract Review","proficiency":2},{"name":"Litigation","proficiency":3}],"experience":[{"company":"Manulife","title":"Engineer","seniority":"mid","start_year":2015,"end_year":null,"is_current":true},{"company":"Hang Seng Bank","title":"Scientist","seniority":"senior","start_year":2017,"end_year":2022,"is_current":true}],"education":[{"school":"City University of Hong Kong","degree":"BSc","field":"Legal","start_year":2017,"end_year":2020}]}', 'id': 'lc_ffb06403-937e-4637-aa60-c1f7ac2b6da9'}]


In [74]:
# Test 8: Retrieve LinkedIn interaction data (network strength, posts, endorsements)
# Replace 456 with a valid person ID from a previous search.

try:
    result = await tools[5].ainvoke({'person_id': 456})
    print("Test 8 - get_linkedin_interactions:")
    print(result)
except Exception as e:
    print("Test 8 - get_linkedin_interactions failed (expected if person_id 456 is invalid):", e)

Test 8 - get_linkedin_interactions:
[{'type': 'text', 'text': '{"profile_id":456,"post_count":4,"total_likes":5,"liked_by":[4390,3622,7500,4269,8464],"engagement_score":1.25}', 'id': 'lc_7e160031-b758-4e3e-84ab-a69a9808c639'}]


In [84]:
#  Evaluation code


def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }

In [85]:
def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5, "scores must have length 5"
    assert len(groundtruth) == 5, "groundtruth must have length 5"

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }

In [86]:
scores = [0.2, 0.3, 0.4, 0.5, 0.6]   # Example scores (replace with your actual results)
groundtruth = [1, 1, 1, 0, 0]        # Provided ground truth, do not modify

result = evaluate(scores, groundtruth)
print(result)

{'decisions': [0, 0, 0, 0, 1], 'correct': 1, 'total': 5, 'final_score': 0.2}
